**Note:**

- run all instuctions below on HOST machine
- remember you continue working within branch `experiments`

# Push data to remote DVC storage (local)

## Create remote storage (local)


```bash
dvc remote add -d local /tmp/dvc/dvc-5-demo-project-iris
git add .
git commit -m "add dvc remote - local"
```

## Push data to remote storage

```bash
dvc push
```


# Build  CI pipeline

## Register gitlab runners

### Get register token

1. go to **Settings** -> **CI / CD** -> **Specific Runners** -> **Set up a specific Runner manually**;
2. copy  registration token from `3.Use the following registration token during setup:`;
3. define env var:

```bash
export REGISTRATION_TOKEN=<registration_token>
```

### shell runner 

```bash
gitlab-runner register \
            --non-interactive \
            -u https://gitlab.com/ \
            -r $REGISTRATION_TOKEN \
            --tag-list shell \
            --executor shell 
```

__Notes__:
* *--non-interactive* - to avoid gitlab-runner prompts;
* *-u* - CI server URL.


### docker runner

```bash
gitlab-runner register \
            --non-interactive \
            -u https://gitlab.com/ \
            -r $REGISTRATION_TOKEN \
            --tag-list docker,experiments \
            --executor docker \
            --docker-image mlrepa/cml:latest \
            --docker-disable-cache \
            --docker-volumes "/tmp:/tmp"
```

__Notes__:
* *--docker-image* - docker image which will be used by default to 
run CI job if another image will not be specified in job definition;
* *--docker-disable-cache* - avoid caching repository and data in container's volume,
volume will not be stored; it guarantees repository is fresh at every `cml` run;
if cache enabled repository can stay outdated;
* *--docker-volumes* - mount folders from host to docker container; 
it gives access to local DVC remote.

## Start runners

```bash
gitlab-runner run 
```

## Create .gitlab-ci.yml file



```yaml
variables:
  DEPLOY_IMAGE: mlrepa/deploy-sklearn:latest
  PROJECT_MODEL_PATH: models/model.joblib
  DOCKER_MODEL_PATH: /home/deploy/models/model.joblib

stages:
  - build
  - test
  - experiments
```

## Create `build` job

```yaml
build:
  stage: build
  tags:
    - shell
  only:
    - master
    - test
  script:
    - docker pull $DEPLOY_IMAGE
    - |
      if [ ! -d "/tmp/$CI_PROJECT_NAME" ]; then
        project_url=${CI_PROJECT_URL//"https://"/}
        git clone \
          https://oauth2:$repo_token@${project_url}.git \
          "/tmp/$CI_PROJECT_NAME"
      fi
      cd /tmp/$CI_PROJECT_NAME
      git checkout $CI_BUILD_REF_NAME
      git pull origin $CI_BUILD_REF_NAME
      dvc fetch
      dvc checkout
```

## Create `test` job

```yaml
test:
  stage: test
  tags:
    - shell
  only:
    - master
    - test
  script:
    - cd /tmp/$CI_PROJECT_NAME
    - |
      docker run \
            --name=deploy-sklearn-test \
            --rm \
            -v $(pwd)/tests:/home/deploy/tests \
            -v $(pwd)/$PROJECT_MODEL_PATH:$DOCKER_MODEL_PATH \
            -e MODEL_PATH=$DOCKER_MODEL_PATH \
                $DEPLOY_IMAGE /bin/bash -c "
                    python /home/deploy/tests/test_model.py
                "
```

## Create `cml` job

```yaml
cml:
    stage: experiments
    image: mlrepa/cml:latest #dvcorg/cml-py3:latest
    only:
      - experiments
    tags:
      - docker,experiments
    script:
      - apt-get install -y jq
      - git fetch
      - git checkout $CI_BUILD_REF_NAME --
      - git pull origin $CI_BUILD_REF_NAME
      - dvc fetch
      - dvc checkout
      - dvc repro
      - |
        echo "## f1_score: $(cat experiments/metrics.json | jq '.f1_score')" >> report.md
      - dvc metrics diff --show-md >> report.md
      - echo >> report.md
      - cml-publish experiments/confusion_matrix.png --md >> report.md
      - cml-send-comment report.md
```

## Full `.gitlab-ci.yaml`

```yaml
variables:
  DEPLOY_IMAGE: mlrepa/deploy-sklearn:latest
  PROJECT_MODEL_PATH: models/model.joblib
  DOCKER_MODEL_PATH: /home/deploy/models/model.joblib

stages:
  - build
  - test
  - experiments

build:
  stage: build
  tags:
    - shell
  only:
    - master
    - test
  script:
    - docker pull $DEPLOY_IMAGE
    - |
      if [ ! -d "/tmp/$CI_PROJECT_NAME" ]; then
        project_url=${CI_PROJECT_URL//"https://"/}
        git clone \
          https://oauth2:$repo_token@${project_url}.git \
          "/tmp/$CI_PROJECT_NAME"
      fi
      cd /tmp/$CI_PROJECT_NAME
      git checkout $CI_BUILD_REF_NAME
      git pull origin $CI_BUILD_REF_NAME
      dvc fetch
      dvc checkout

test:
  stage: test
  tags:
    - shell
  only:
    - master
    - test
  script:
    - cd /tmp/$CI_PROJECT_NAME
    - |
      docker run \
            --name=deploy-sklearn-test \
            --rm \
            -v $(pwd)/tests:/home/deploy/tests \
            -v $(pwd)/$PROJECT_MODEL_PATH:$DOCKER_MODEL_PATH \
            -e MODEL_PATH=$DOCKER_MODEL_PATH \
                $DEPLOY_IMAGE /bin/bash -c "
                    python /home/deploy/tests/test_model.py
                "

cml:
    stage: experiments
    image: mlrepa/cml:latest #dvcorg/cml-py3:latest
    only:
      - experiments
    tags:
      - docker,experiments
    script:
      - apt-get install -y jq
      - git fetch
      - git checkout $CI_BUILD_REF_NAME --
      - git pull origin $CI_BUILD_REF_NAME
      - dvc fetch
      - dvc checkout
      - dvc repro
      - |
        echo "## f1_score: $(cat experiments/metrics.json | jq '.f1_score')" >> report.md
      - dvc metrics diff --show-md >> report.md
      - echo >> report.md
      - cml-publish experiments/confusion_matrix.png --md >> report.md
      - cml-send-comment report.md

```

## Push commits to GitLab repo 


### trigger `build` and `test` jobs

push to `master`

```bash
git checkout master
git merge experiments
git push origin master
```

or push to `test` for testing:

* create branch `test` from `experiments`

```bash
git checkout -b test
git push origin test
```

* of merge `experiments` to `test` if `test` already exists:

```bash
git checkout test
git merge experiments
git push origin test
```

### trigger `cml` job

```bash 
git push origin experiments
```